In [6]:
!pip install huggingface-hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `arshia` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pu

In [8]:
!pip install datasets transformers torch


In [22]:
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from transformers import AutoTokenizer

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

# Transformer NLI Model
class TransformerNLI(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=4, num_classes=3, max_len=128):
        super(TransformerNLI, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dim_feedforward=512,
            dropout=0.1
        )
        self.fc = nn.Linear(d_model, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask=None):
        # Embedding and positional encoding
        embeddings = self.embedding(input_ids)
        embeddings = self.positional_encoding(embeddings)

        # Transformer Encoder
        embeddings = embeddings.permute(1, 0, 2)
        output = self.transformer(embeddings, embeddings)
        output = output.mean(dim=0)

        logits = self.fc(output)
        return logits

snli = load_dataset("snli")


snli = snli.filter(lambda example: 0 <= example['label'] <= 2)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Encode the dataset
def encode_data(batch):
    encoding = tokenizer(batch['premise'], batch['hypothesis'], padding='max_length', truncation=True, max_length=128)
    batch['input_ids'] = encoding['input_ids']
    batch['label'] = batch['label']
    return batch

snli = snli.map(encode_data, batched=True)

snli.set_format(type='torch', columns=['input_ids', 'label'])

train_subset = snli['train']
val_subset = snli['validation']
test_subset = snli['test']

train_loader = DataLoader(train_subset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=16)
test_loader = DataLoader(test_subset, batch_size=16)


vocab_size = tokenizer.vocab_size
model = TransformerNLI(vocab_size, d_model=256, nhead=8, num_layers=4, num_classes=3, max_len=128)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        labels = batch['label'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}, Loss: {avg_loss:.4f}")


def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    total_eval_loss = 0

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['label'].to(device)

            logits = model(input_ids)
            loss = criterion(logits, labels)
            total_eval_loss += loss.item()

            predictions = logits.argmax(dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_eval_loss / len(loader)
    accuracy = correct / total
    return avg_loss, accuracy

val_loss, val_accuracy = evaluate(val_loader)
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy * 100:.2f}%")

test_loss, test_accuracy = evaluate(test_loader)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1, Loss: 1.0156
Epoch 2, Loss: 0.9008
Epoch 3, Loss: 0.8395
Epoch 4, Loss: 0.7939
Epoch 5, Loss: 0.7486
Epoch 6, Loss: 0.7057
Epoch 7, Loss: 0.6637
Epoch 8, Loss: 0.6236
Epoch 9, Loss: 0.5828
Epoch 10, Loss: 0.5427
Validation Loss: 0.8431, Validation Accuracy: 68.10%
Test Loss: 0.8519, Test Accuracy: 67.86%


In [4]:
%pip install tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 5.8 MB/s eta 0:00:00
